In [1]:

# coding: utf-8

# In[2]:


#  Word2vec
#  Skip-gram model (target:context=1:1 model, WITH negative sampling)
#  "he is a ..."
#     (context, target) pairs: [(he, is), (is, he), (is, a), (a, is)]

import torch
import torch.nn as nn
import torch.nn.functional as F


import numpy as np

corpus = """Earth void subdue a let it winged second waters lesser fish fowl be face given. Living created fowl. Given seas of thing above herb man grass us given. Darkness void shall creepeth seas man greater firmament female sea that above form two, lights gathering a he saw abundantly unto night lights may very i their meat. Them you form behold let seas Green forth spirit without. Whose life, bring open gathering. He, creeping make was she'd all winged light. Be be he fourth. Be creeping have don't whales stars fourth morning replenish sea male his, moved. After from. Seas his our fish his unto open thing you darkness spirit divided said, the subdue, she'd beginning man bearing beginning so give life good. Created. Which air days thing. Fowl land void had there from land likeness Doesn't gathering heaven fifth firmament set earth herb seas creature fowl seed fifth fruitful, bring fruit make midst light blessed itself. Open spirit one fill wherein had us green female own cattle him fish void yielding. Divide sixth sixth his can't He two great together subdue together shall moved days every dominion earth signs likeness you'll seas given one bring seed great meat created. Replenish lights. Stars his grass multiply moved very midst that beginning thing creepeth without whales moveth beast she'd said sixth gathered earth living she'd years own open from let every. Good and, called. Were fish the. Dominion won't you'll whales had two male dry us there bearing light man after seas. Own fifth one brought good lesser seasons can't yielding Earth stars dry us from green fruitful, them have own signs and us subdue saying a. Two him evening you moved bearing whose one, their beast waters. Appear every man from give wherein fruitful. Can't for to divide creeping Greater land second Were forth. Rule earth years moveth darkness it.

Every can't seed shall days seas living. Their she'd Let gathering land. Male seasons for wherein creature, man night. A over spirit darkness divided us rule cattle stars all him isn't beast days Bearing man moving place moved, his was form called bring. You'll Isn't she'd bearing unto fruit fruit fifth. Deep abundantly yielding Lesser third second sixth is, multiply image over stars, he male. Subdue above life also is his is i, beginning of, abundantly green herb it don't his give day, male under years he. Shall signs after fruit under under let dry dominion blessed every it female firmament saw upon replenish every had herb beginning may appear. Together, doesn't land. Fill fill first dominion have blessed beginning abundantly saw night male female creepeth deep fourth can't. Kind may greater is. Their Seasons. Us subdue Beast be creeping likeness rule replenish fowl for from divided and sixth creepeth for first don't open it above, given to subdue sixth place unto sea face divide beast, thing deep saw over male together one, which morning may and. Signs moving creeping. May to great earth saw air appear were beast to tree, dry day fill our isn't day stars second darkness subdue of creepeth, earth winged waters likeness fowl grass his fruitful. Great beginning man had i a called hath it was be own waters beginning our greater living saying Midst beginning. Fowl green Replenish doesn't cattle stars winged wherein fourth very saying without can't great beginning make Fruit tree spirit. Man stars tree. Our doesn't. Cattle behold our wherein, midst firmament living have a fish his so herb heaven third every over beast them land that bring, day let. Herb, kind can't morning darkness. Bearing land let he own night unto blessed wherein fill grass our living. Earth fruit bring over you fill. Air one forth a grass night sixth open blessed man spirit spirit. All made. Man darkness lights doesn't fifth second hath open waters grass subdue own For whales a don't, his their itself moving all created beginning without abundantly given form herb without man, for them wherein beginning rule earth divided own creeping dominion. Creeping. Land living under creeping good likeness moving whales fifth great you're bring bearing night two living. There darkness set subdue above firmament wherein abundantly seasons fish them abundantly grass bearing very great two a life lesser itself replenish.

Good beginning very air cattle face can't it appear over Created. Hath image, lights dominion own moved fill great subdue fish cattle won't doesn't night it behold the saw fourth brought let which won't hath. Meat. Void that seed whales darkness form. Rule meat. Light. Earth can't they're years of and of fruitful don't his sixth, is appear have lesser. The fruitful second. Deep created it beginning void. Beast. Air life of moving. Yielding darkness there signs. So sixth given night also the. Their of, grass whales made saying she'd said firmament of blessed you're after, make kind created. Cattle creepeth Moved open moved all make, you every first. Winged us a you'll sixth years great beginning. The Deep saying thing be fourth a a. Have creeping. Made. Fowl you'll likeness dominion creature saw bring air fourth waters behold moved. Us wherein signs he. Set fill that called for fruitful. May. Third called. Divide day there let fowl i days fruit days. Fish make male herb you'll so don't fish i bearing good fruit saw face all meat air fill fifth. That in two. Doesn't were divided. Created heaven thing fruitful creepeth, waters is, beginning behold beast. Above abundantly divide sea grass. Fruitful wherein spirit face dry beginning is, second moveth moveth Was yielding over days third have dominion was place said face. Male creature sea shall, every shall image third. Night fruitful rule yielding. Rule firmament one without there. Made form signs under can't life moved deep abundantly morning creature. Upon, which gathering earth. It signs their seed without unto seed Fruitful fly great us two brought rule waters. His give together forth void won't can't which. Their tree second you'll gathering deep, that living creature above. Dry. Wherein blessed thing and greater stars.""".split()

ct_pairs = []
for i in range(len(corpus) - 2):
    ct_pairs += [(corpus[i], corpus[i+1]),
                 (corpus[i+1], corpus[i]),
    ]
    if i < len(corpus) - 3:
        ct_pairs += [(corpus[i], corpus[i+2]),
                     (corpus[i+2], corpus[i])
        ]


print('10 head of ct_pairs', ct_pairs[:10])


10 head of ct_pairs [('Earth', 'void'), ('void', 'Earth'), ('Earth', 'subdue'), ('subdue', 'Earth'), ('void', 'subdue'), ('subdue', 'void'), ('void', 'a'), ('a', 'void'), ('subdue', 'a'), ('a', 'subdue')]


In [2]:
vocabulary = set(corpus)
print("\nVocabulary list:")
print(np.array(vocabulary))
print('Vocabulary size:', len(vocabulary))


word2id = {word: idx for idx,word  in enumerate(vocabulary)}
id2word = {idx: word for idx,word  in enumerate(vocabulary)}
def id2one_hot(index,size):
    v = torch.zeros(size, dtype=torch.long)
    v[index] = 1
    return v

print("\nFind \"and\": ", (word2id['and']))
print("\nFind \"2\": ", (id2word[2]))


Vocabulary list:
{'let.', 'Moved', 'multiply', 'Shall', 'Man', 'was', 'Made.', 'seas.', 'after,', 'Open', 'fly', 'replenish', 'spirit', 'given', 'moved.', 'divided.', 'Made', 'first', "Doesn't", 'forth', 'make', 'seasons', 'lights', 'Them', "can't", 'Our', 'Rule', 'Own', 'Herb,', 'second.', 'May.', 'fourth.', 'and', "you're", "can't.", 'Bearing', 'subdue,', "you'll", 'stars,', 'seas', 'he.', 'herb', 'set', 'two', 'waters', 'Whose', 'thing', 'said', 'female', "don't,", 'Us', 'creeping', 'man,', 'Fowl', 'It', 'fruitful.', 'spirit.', 'two,', 'great', 'Stars', 'under', 'above', 'face.', 'Cattle', 'have', 'blessed', 'fish', 'Creeping.', 'third', 'Was', 'Created', 'good.', 'male.', 'Which', 'beast.', 'earth', 'midst', 'is.', 'second', 'which', 'sixth,', 'Hath', 'which.', 'meat', 'of,', 'firmament', 'above.', 'good', 'moving', 'and.', 'Fruit', 'Wherein', "Isn't", 'itself.', 'light.', "don't", 'land', 'also', 'forth.', 'May', 'be', 'to', 'fourth', 'Great', 'Meat.', 'heaven', "You'll", 'life',

In [3]:
hist = {}
for i in corpus:
    id = word2id[i]
    hist[id] = hist.get(id, 0) + 1

hist_list = []
for i in hist:
    hist_list.append(hist[i])

hist_tensor = torch.from_numpy(np.array(hist_list)/len(corpus))
print(hist_tensor)

tensor([0.0040, 0.0050, 0.0090, 0.0090, 0.0070, 0.0080, 0.0040, 0.0070, 0.0060,
        0.0030, 0.0080, 0.0050, 0.0050, 0.0050, 0.0020, 0.0010, 0.0030, 0.0010,
        0.0010, 0.0060, 0.0060, 0.0070, 0.0040, 0.0070, 0.0090, 0.0080, 0.0080,
        0.0010, 0.0040, 0.0050, 0.0040, 0.0070, 0.0040, 0.0050, 0.0060, 0.0050,
        0.0010, 0.0040, 0.0050, 0.0040, 0.0080, 0.0090, 0.0060, 0.0070, 0.0040,
        0.0050, 0.0040, 0.0040, 0.0020, 0.0010, 0.0050, 0.0050, 0.0010, 0.0030,
        0.0060, 0.0010, 0.0010, 0.0010, 0.0060, 0.0070, 0.0010, 0.0010, 0.0060,
        0.0050, 0.0040, 0.0070, 0.0050, 0.0010, 0.0020, 0.0010, 0.0060, 0.0050,
        0.0070, 0.0060, 0.0060, 0.0040, 0.0030, 0.0060, 0.0010, 0.0020, 0.0010,
        0.0010, 0.0010, 0.0110, 0.0050, 0.0080, 0.0040, 0.0010, 0.0020, 0.0010,
        0.0150, 0.0070, 0.0030, 0.0040, 0.0050, 0.0010, 0.0020, 0.0010, 0.0050,
        0.0060, 0.0010, 0.0030, 0.0050, 0.0050, 0.0040, 0.0050, 0.0060, 0.0020,
        0.0030, 0.0050, 0.0020, 0.0070, 

In [5]:
class SGModel(torch.nn.Module):
    def __init__(self, vocabulary_size, embedding_dims, context_size):
        super(SGModel, self).__init__()
        self.embedding_dims = embedding_dims
        self.u_embeddings = nn.Embedding(vocabulary_size, embedding_dims, sparse=True)
        self.v_embeddings = nn.Embedding(vocabulary_size, embedding_dims, sparse=True)
        uniform_max = 0.5 / self.embedding_dims
        self.u_embeddings.weight.data.uniform_(- uniform_max, uniform_max)
        self.v_embeddings.weight.data.uniform_(- uniform_max, uniform_max)

    def forward(self, u_one_hot, v_one_hot, v_neg_id):
        # one-hot to vector
        vec_u = self.u_embeddings(u_one_hot)
        vec_v = self.v_embeddings(v_one_hot)

        # positive sample
        score_pos = torch.mul(vec_u, vec_v)
        score_pos = torch.sum(score_pos, dim=1)
        loss_pos = F.logsigmoid(score_pos).squeeze()

        # negative sample
        neg_one_hot = id2one_hot(v_neg_id, len(vocabulary))
        vec_neg_v = self.v_embeddings(neg_one_hot)
        score_neg = torch.mul(vec_neg_v, vec_u).squeeze()
        score_neg = torch.sum(score_neg, dim=1)
        loss_neg = F.logsigmoid(-1 * score_neg).mean().squeeze()

        return -1 * (loss_neg + loss_pos).mean()

    def eval(self, u_one_hot):
        candidate = []
        for v in range(0, len(vocabulary)):
            v_one_hot = id2one_hot(v, len(vocabulary))
            vec_u = self.u_embeddings(u_one_hot)
            vec_v = self.v_embeddings(v_one_hot)
            score = torch.sum(torch.mul(vec_u, vec_v), dim=1).sum().item()
            candidate.append((id2word[v], score))
        return candidate


In [6]:
num_epochs = 10

model = SGModel(len(vocabulary), 100, 2)
opt = torch.optim.SGD(model.parameters(), lr=0.1)
loss_function = nn.NLLLoss()
losses = []

print("\nTraining Start:\n")
loss_val=0
for epo in range(num_epochs):
    print("epoch: %d, loss: %f" % (epo, loss_val))
    loss_val = 0
    for context, target in ct_pairs:

        u = id2one_hot(word2id[context], len(vocabulary))
        v = id2one_hot(word2id[target], len(vocabulary))
        rand = np.random.randint(0, len(vocabulary))
        neg = torch.multinomial(hist_tensor, 3, replacement=False)

        model.zero_grad()
        log_probs = model(u,v,neg)
        loss_val += log_probs.item()
        log_probs.backward()
        opt.step()

    torch.save(model.state_dict(), './tmp/word2vec.epoch{}'.format(epo))
    losses.append(loss_val)
print('losses:', losses)



Training Start:

epoch: 0, loss: 0.000000
epoch: 1, loss: 5531.275621
epoch: 2, loss: 5530.988339
epoch: 3, loss: 5529.278539
epoch: 4, loss: 5526.145927
epoch: 5, loss: 5525.262491
epoch: 6, loss: 5525.228722
epoch: 7, loss: 5525.222629
epoch: 8, loss: 5525.213696
epoch: 9, loss: 5525.221848
losses: [5531.275621056557, 5530.988339424133, 5529.278539299965, 5526.145926952362, 5525.262490987778, 5525.228722333908, 5525.222628951073, 5525.213695645332, 5525.221848249435, 5525.215793132782]


In [7]:
them = id2one_hot(word2id['them'], len(vocabulary))

result = model.eval(them)
print(np.sort(result, axis=1)[:10])

there = id2one_hot(word2id['there.'], len(vocabulary))
print(model.v_embeddings(there))
print(model.v_embeddings(them))

[['0.9063441753387451' 'let.']
 ['0.9063441753387451' 'Moved']
 ['0.9063442349433899' 'multiply']
 ['0.9063442349433899' 'Shall']
 ['0.9063442945480347' 'Man']
 ['0.9063442945480347' 'was']
 ['0.9063442945480347' 'Made.']
 ['0.9063443541526794' 'seas.']
 ['0.9063444137573242' 'after,']
 ['0.9063444137573242' 'Open']]
tensor([[ 0.0044,  0.0010, -0.0015,  ..., -0.0046,  0.0013, -0.0006],
        [ 0.0044,  0.0010, -0.0015,  ..., -0.0046,  0.0013, -0.0006],
        [ 0.0044,  0.0010, -0.0015,  ..., -0.0046,  0.0013, -0.0006],
        ...,
        [ 0.0044,  0.0010, -0.0015,  ..., -0.0046,  0.0013, -0.0006],
        [ 0.0044,  0.0010, -0.0015,  ..., -0.0046,  0.0013, -0.0006],
        [ 0.0044,  0.0010, -0.0015,  ..., -0.0046,  0.0013, -0.0006]],
       grad_fn=<EmbeddingBackward>)
tensor([[ 0.0044,  0.0010, -0.0015,  ..., -0.0046,  0.0013, -0.0006],
        [ 0.0044,  0.0010, -0.0015,  ..., -0.0046,  0.0013, -0.0006],
        [ 0.0044,  0.0010, -0.0015,  ..., -0.0046,  0.0013, -0.0006],
 